In [1]:
import json
import pandas as pd
import numpy as np
import torch
import nltk
from nltk import TweetTokenizer
from nltk.corpus import stopwords
import string
import numpy as np
import json
nltk.download('stopwords')

/Users/johnschool/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/johnschool/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def get_file(filename):
    data = {}
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print("JSON data loaded successfully:")
    except Exception as e:
        print(e)
    return {"data": data}

In [ ]:
node_new_Twibot22 = get_file("data/Twibot-22/node_new.json")
label_new_Twibot22 = get_file("data/Twibot-22/label_new.json")

JSON data loaded successfully:
JSON data loaded successfully:


In [7]:
point1 = node_new_Twibot22["data"]["t1498018021658431488"]
point1_author_id = point1["author_id"]

In [8]:
point1_author_id

20441260

In [9]:
# the tweet
print(point1)
# is the author at key "u" + author_id a bot or not? 
label_new_Twibot22["data"]["u"+str(point1_author_id)]

{'attachments': None, 'author_id': 20441260, 'context_annotations': None, 'conversation_id': 1498018021658431488, 'created_at': '2022-02-27 19:31:42+00:00', 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'rhpsia', 'name': 'Sia At The Game', 'id': 809332867, 'id_str': '809332867', 'indices': [3, 10]}], 'urls': []}, 'geo': None, 'id': 't1498018021658431488', 'in_reply_to_user_id': None, 'lang': 'en', 'possibly_sensitive': False, 'public_metrics': {'retweet_count': 10, 'reply_count': None, 'like_count': 0, 'quote_count': None}, 'referenced_tweets': None, 'reply_settings': None, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'text': 'RT @rhpsia: BREAKING: Telecom giant Ericsson used slush funds, trips for Iraqi officials &amp; payoffs through middlemen to executives — and po…', 'withheld': None}


'human'

In [ ]:
# Count top-level keys in data/Twibot-22/node_new.json that start with 't'
try:
    # node_new_Twibot22 is expected to be a dict returned by get_file() earlier in this notebook
    data_obj = node_new_Twibot22 if not isinstance(node_new_Twibot22, dict) else node_new_Twibot22.get("data", node_new_Twibot22)
except NameError:
    # If the variable isn't defined (cell not run), load the file now
    print("node_new_Twibot22 not found in the notebook state - loading from file")
    node_new_Twibot22 = get_file("data/Twibot-22/node_new.json")
    data_obj = node_new_Twibot22.get("data", node_new_Twibot22)

# Ensure we have a mapping-like object
if not isinstance(data_obj, dict):
    print("Unexpected format: expected a JSON object/dict for node data.")
else:
    keys = [k for k in data_obj.keys() if isinstance(k, str) and k.startswith("t")]
    count = len(keys)
    print(f"Found {count} keys starting with 't' in data/Twibot-22/node_new.json")
    print(f"Scanned {len(data_obj)} top-level keys in the node object.")
    # Show up to 10 sample keys to verify
    print("Sample keys (up to 10):", keys[:10])

Found 31228 keys starting with 't' in Twibot-22/node_new.json
Scanned 34936 top-level keys in the node object.
Sample keys (up to 10): ['t1498018021658431488', 't1497947823005962245', 't1497902014763786241', 't1497736000008867841', 't1497640323631222784', 't1497417782450864130', 't1497402632301854722', 't1497288192969654273', 't1497277930623361033', 't1497275497126019076']


In [11]:
# add a label to every tweet key
num_bot = 0 
num_human = 0
num_unlabeled = 0

for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    
    if "u"+str(cur_author_id) in label_new_Twibot22["data"]: 
        cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    else:
        num_unlabeled+= 1 # ids nonexistent in labels

    if (cur_author_label == "human"):
        num_human += 1
    if (cur_author_label == "bot"):
        num_bot += 1

    node_new_Twibot22["data"][cur_tweet_id]["author_label"] = cur_author_label

print(num_bot)
print(num_human)
print(num_unlabeled)

8171
23057
644


There are 8171 bot tweets, and 23057 human tweets


In [12]:
# from pyspark.ml.feature import Tokenizer, CountVectorizer
# from pyspark.ml.classification import LogisticRegression

# tokenizer = Tokenizer(inputCol="text", outputCol="words")
# cv = CountVectorizer(inputCol="words", outputCol="features")
# lr = LogisticRegression(featuresCol="features", labelCol="label")

In [ ]:
# This code is for John Ryu because he is a meanie
complete_dict = {}

for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]:
        continue
    
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    text = cur_tweet["text"]
    
    complete_dict[cur_tweet_id] = {
        "label": cur_author_label,
        "text": text
    }

with open("data/tweets_with_labels.json", "w", encoding="utf-8") as f:
    json.dump(complete_dict, f, ensure_ascii=False, indent=4)

In [14]:
tknzr = TweetTokenizer(preserve_case =False)
stops = set(stopwords.words("english"))
stops.update(string.punctuation)
set_1 = {'…', '’', '...', '“', '”','‘'}
stops.update(set_1)


In [15]:
print(tknzr.tokenize(node_new_Twibot22["data"][keys[0]]["text"]))
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])

    


['rt', '@rhpsia', ':', 'breaking', ':', 'telecom', 'giant', 'ericsson', 'used', 'slush', 'funds', ',', 'trips', 'for', 'iraqi', 'officials', '&', 'payoffs', 'through', 'middlemen', 'to', 'executives', '—', 'and', 'po', '…']
Top 100 bot tokens:
[('rt', 2695), ('new', 366), ('today', 331), ('like', 281), ('us', 259), ('get', 257), ('one', 254), ('de', 239), ('#zerotrust', 238), ('2021', 210), ('3', 206), ('️', 205), ('punk', 201), ('time', 200), ('dos', 198), ('price', 196), ('💢', 195), ('【', 187), ('@snaptadc', 172), ('people', 164), ('🔥', 164), ('join', 163), ('check', 158), ('2022', 158), ('】', 157), ('great', 156), ('la', 156), ('#cryptoart', 154), ('know', 152), ('first', 150), ('make', 147), ('good', 142), ('#cybersecurity', 142), ('day', 139), ('trust', 139), ('#nft', 137), ('year', 136), ('pips', 134), ('love', 133), ('1', 129), ('see', 128), ('#ztna', 128), ('work', 126), ('follow', 125), ('zero', 124), ('#nftgiveaway', 124), ('please', 123), ('security', 122), ('win', 122), ('#

In [16]:
bot_lengths = np.array([])
human_lengths = np.array([])
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    if (cur_author_label == "human"):
        human_lengths = np.append(human_lengths, len(tokenized_tweet))
    if (cur_author_label == "bot"):
        bot_lengths = np.append(bot_lengths, len(tokenized_tweet))
def stats(arr):
    mean = np.mean(arr)
    std = np.std(arr)
    q2 = np.percentile(arr, 50) 
    minimum = np.min(arr)
    maximum = np.max(arr)

    print("Mean:", mean)
    print("Standard deviation:", std)
    print("Median (Q2):", q2)
    print("Minimum:", minimum)
    print("Maximum:", maximum)

print("Human tweet length stats:")
stats(human_lengths)
print("Bot tweet length stats:")
stats(bot_lengths)

Human tweet length stats:
Mean: 24.156702777406267
Standard deviation: 14.07049034910416
Median (Q2): 24.0
Minimum: 1.0
Maximum: 89.0
Bot tweet length stats:
Mean: 22.005519440696677
Standard deviation: 12.500826670200132
Median (Q2): 21.0
Minimum: 1.0
Maximum: 77.0


In [17]:
from nltk import bigrams
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    filtered_tweet = bigrams(filtered_tweet)
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])

Top 100 bot tokens:
[(('dos', 'punk'), 188), (('rt', '@snaptadc'), 169), (('#nftcommumity', '#nftgiveaway'), 120), (('#nftdrop', '#nftartist'), 120), (('#nftartist', '#nftcollector'), 120), (('#nftcollector', '#nftproject'), 120), (('zero', 'trust'), 117), (('#forex', '#signaltrading'), 107), (('#signaltrading', '#trading'), 107), (('#trading', '#eurusd'), 107), (('#eurusd', '#foreignexchange'), 107), (('simple', 'steps'), 88), (('get', 'iphone'), 87), (('iphone', 'xs'), 87), (('xs', 'today'), 87), (('today', 'follow'), 87), (('follow', 'simple'), 87), (('steps', 'win'), 87), (('win', 'click'), 87), (('click', 'submit'), 87), (('submit', 'e-mail'), 87), (('e-mail', 'lucky'), 87), (('lucky', 'winner.https'), 87), (('winner.https', ':/'), 87), (('#nftart', '#cryptoart'), 84), (('join', 'us'), 82), (('#nftgiveaway', '#nfts'), 71), (('price', '😊'), 70), (('😊', '3'), 70), (('3', 'tezos'), 70), (('tezos', '🌼'), 70), (('🌼', '#nftcommumity'), 70), (('#nfts', '#tezos'), 70), (('#tezos', '#nftdr

In [18]:
from nltk import trigrams
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    filtered_tweet = trigrams(filtered_tweet)
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])

Top 100 bot tokens:
[(('#nftdrop', '#nftartist', '#nftcollector'), 120), (('#nftartist', '#nftcollector', '#nftproject'), 120), (('#forex', '#signaltrading', '#trading'), 107), (('#signaltrading', '#trading', '#eurusd'), 107), (('#trading', '#eurusd', '#foreignexchange'), 107), (('get', 'iphone', 'xs'), 87), (('iphone', 'xs', 'today'), 87), (('xs', 'today', 'follow'), 87), (('today', 'follow', 'simple'), 87), (('follow', 'simple', 'steps'), 87), (('simple', 'steps', 'win'), 87), (('steps', 'win', 'click'), 87), (('win', 'click', 'submit'), 87), (('click', 'submit', 'e-mail'), 87), (('submit', 'e-mail', 'lucky'), 87), (('e-mail', 'lucky', 'winner.https'), 87), (('lucky', 'winner.https', ':/'), 87), (('price', '😊', '3'), 70), (('😊', '3', 'tezos'), 70), (('3', 'tezos', '🌼'), 70), (('tezos', '🌼', '#nftcommumity'), 70), (('🌼', '#nftcommumity', '#nftgiveaway'), 70), (('#nftcommumity', '#nftgiveaway', '#nfts'), 70), (('#nftgiveaway', '#nfts', '#tezos'), 70), (('#nfts', '#tezos', '#nftdrop'), 